# Random Forest Model
## 변수 처리 방법
- Passenger ID : 제거
- Pclass : Ordinal 변수이므로 그대로 채용.
- Sex : Label-Encode
- Age : 그대로 (Mean으로 결측값 Impute)
- Sibsp, Parch : 그대로
- Ticket : 제거
- Fare : 그대로, 단 Ticket변수와 Sibsp+parch 사용하여 그룹멤버들만큼 나눠서 1인당 요금으로 계산
- Cabin : Binary로 변환
- Embarked : one-hot encoding with NaN imputed as 'C'. (NaN승객들과 같은 Pclass 승객들의 Fare mean으로 C인것을 유추)

## 생성변수
- Group : Ticket 변수가 unique하지 않거나 Sibsp+parch > 0 일때 1, 아닐때 0을 가지는 단체승객여부 변수.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Import Dataset
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

#결측치 처리 (Age, Embarked)
df_train['Age'].fillna(df_train['Age'].mean(),inplace=True)
df_test['Age'].fillna(df_train['Age'].mean(),inplace=True)
df_train['Embarked'].fillna('C',inplace=True)

#Cabin 결측치 처리 겸 변수 변환
df_train['Cabin'] = df_train.Cabin.apply(lambda x: 1 if pd.notnull(x) else 0)
df_test['Cabin'] = df_test.Cabin.apply(lambda x: 1 if pd.notnull(x) else 0)

#Group 변수 생성
#Family size랑 dup_count 중 큰걸로 fare 나누기.
df_train['Family_size'] = df_train['SibSp']+df_train['Parch']+1
df_train['dup_count'] = df_train.groupby(['Ticket'])['Ticket'].transform('count')
df_train['Group'] = ~((df_train['Family_size'] == 1) & (df_train['dup_count'] == 1))
df_train['Group_count'] = df_train[['Family_size','dup_count']].max(axis=1)
df_train['Fare_ind'] = df_train['Fare']/df_train['Group_count']

#Fare 결측치 처리 in test
df_test['Fare'].fillna(df_test['Fare'].mean(),inplace=True)

df_test['Family_size'] = df_test['SibSp']+df_test['Parch']+1
df_test['dup_count'] = df_test.groupby(['Ticket'])['Ticket'].transform('count')
df_test['Group'] = ~((df_test['Family_size'] == 1) & (df_test['dup_count'] == 1))
df_test['Group_count'] = df_test[['Family_size','dup_count']].max(axis=1)
df_test['Fare_ind'] = df_test['Fare']/df_test['Group_count']

# Categorical Feature Encoding
df_train['Sex'] = df_train['Sex'].apply(lambda x: 1 if x is 'male' else 0)
df_test['Sex'] = df_test['Sex'].apply(lambda x: 1 if x is 'male' else 0)
df_train = pd.get_dummies(df_train, columns = ['Embarked'],drop_first=True,prefix='', prefix_sep='')
df_test = pd.get_dummies(df_test, columns = ['Embarked'],drop_first=True,prefix='', prefix_sep='')

In [3]:
#모델에 사용하지 않을 변수 제거
train_X = df_train.drop(['PassengerId','Survived','Name','Ticket','Fare','Cabin','Family_size','dup_count','Group_count'],axis=1)
test_X = df_test.drop(['PassengerId','Name','Ticket','Fare','Cabin','Family_size','dup_count','Group_count'],axis=1)
train_y = df_train.Survived

In [4]:
rf = RandomForestClassifier(random_state=0)
parameters = {'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300]}
clf = RandomizedSearchCV(rf, parameters,n_iter = 100, cv=5)
clf.fit(train_X, train_y)

/Users/danielhan/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [5]:
clf.best_params_

{'n_estimators': 100,
 'min_samples_split': 10,
 'min_samples_leaf': 5,
 'max_features': 2,
 'max_depth': 80}

In [6]:
clf.best_score_

0.7283950617283951